In [1]:
from shapely.geometry import Polygon
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import tifffile as tiff
import cv2 as cv
from aeronet.dataset import polygonize, rasterize, BandCollection, parse_directory, FeatureCollection
import os
from scipy import ndimage as ndi
from skimage.segmentation import watershed, mark_boundaries, find_boundaries
from skimage.morphology import binary_dilation
from skimage.exposure import equalize_adapthist, equalize_hist
from skimage.filters import gaussian, threshold_mean
from skimage.feature import peak_local_max

In [2]:
# OUTPUT_PATH = '/home/user/data/ilim_ortho/West 48 d1/polygon/'
# OUTPUT_PATH_RESIZE_05 = '/home/user/data/ilim_ortho/West 48 d1/resize_05m_48d1/'
# OUTPUT_PATH_RESIZE_03 = '/home/user/data/ilim_ortho/West 48 d1/resize_03m_48d1/'
# INPUT_PATH = '/home/user/data/ilim_ortho/West 48 d1/'
# IMAGE = 'West 48 d1.tif'
# IMAGE_DEM = 'West 48 d1_DEM.tif'
# GEOJSON = 'West48_polygon.geojson'

In [3]:
folder_name = '27667'

In [4]:
OUTPUT_PATH = '/home/user/data/provider_drone/' + folder_name + '/polygon/'
OUTPUT_PATH_RESIZE_05 = '/home/user/data/provider_drone/' + folder_name + '/resize_05m_' + folder_name + '/'
OUTPUT_PATH_RESIZE_03 = '/home/user/data/provider_drone/' + folder_name + '/resize_03m_' + folder_name + '/'
INPUT_PATH = '/home/user/data/provider_drone/' + folder_name + '/'
IMAGE = folder_name + '.tif'
IMAGE_DEM = folder_name + '_DEM.tif'
GEOJSON = folder_name + '_polygon.geojson'

if not os.path.isdir(OUTPUT_PATH_RESIZE_03):
    os.makedirs(OUTPUT_PATH_RESIZE_03)
if not os.path.isdir(OUTPUT_PATH_RESIZE_05):
    os.makedirs(OUTPUT_PATH_RESIZE_05)

## Split RGB image into channels, cut the polygon

In [5]:
# Обрезка растрового файла
def pix_from_coords(x, y, transform, size):
    return int(round((x - transform[2])/transform[0])),  int(round((y - transform[5])/transform[4] ))

def calc_raster_boundaries(xmin, xmax, ymin, ymax, transform, size):
    x1, y1 = pix_from_coords(xmin, ymax, transform, size)
    x2, y2 = pix_from_coords(xmax, ymin, transform, size)
    
    x1 = min(max(0, x1), size[1])
    y1 = min(max(0, y1), size[0])
    x2 = min(max(0, x2), size[1])
    y2 = min(max(0, y2), size[0])
    
    if y1 == y2 or x1 == x2:
        return 0,0,0,0
    return x1, y1, x2-x1, y2-y1

def subsample(INPUT_PATH, OUTPUT_PATH, IMAGE_DEM, feat:Polygon):
    bc = BandCollection(parse_directory(INPUT_PATH, [IMAGE_DEM]))
    xmin, ymin, xmax, ymax = feat.shape.bounds
    print(xmin, ymin, xmax, ymax)
    x1, y1, width, height = calc_raster_boundaries(xmin, xmax, ymin, ymax, bc[0].transform, bc[0].shape)
    print(width, height)
    sample = bc.sample(x=x1, y=y1, width=width, height=height)
    sample.save(OUTPUT_PATH)
    del bc

In [6]:
with rasterio.open(INPUT_PATH + IMAGE) as src:
    img = src.read().transpose(1, 2, 0)
    profile = src.profile
profile.update(count=1)

with rasterio.open(INPUT_PATH + 'RED.tif', 'w', **profile) as dst:
    dst.write(img[:,:,0], 1)

with rasterio.open(INPUT_PATH + 'GRN.tif', 'w', **profile) as dst:
    dst.write(img[:,:,1], 1)

with rasterio.open(INPUT_PATH + 'BLU.tif', 'w', **profile) as dst:
    dst.write(img[:,:,2], 1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RasterioDeprecationWarning: Creation keywords stored on datasets by Rasterio versions < 1.0b1 will always be ignored in version 1.0. You may opt in to ignoring them now by setting RIO_IGNORE_CREATION_KWDS=TRUE in your environment.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
subsample(INPUT_PATH, OUTPUT_PATH, 'RED', FeatureCollection.read(INPUT_PATH + GEOJSON)[0])
subsample(INPUT_PATH, OUTPUT_PATH, 'GRN', FeatureCollection.read(INPUT_PATH + GEOJSON)[0])
subsample(INPUT_PATH, OUTPUT_PATH, 'BLU', FeatureCollection.read(INPUT_PATH + GEOJSON)[0])

313037.0867231503 6449308.249773085 313655.60598492646 6449926.769034862
20449 20447
313037.0867231503 6449308.249773085 313655.60598492646 6449926.769034862
20449 20447
313037.0867231503 6449308.249773085 313655.60598492646 6449926.769034862
20449 20447


## Cut the DEM data

In [8]:
with rasterio.open(INPUT_PATH + IMAGE_DEM) as src_DEM:
    img_DEM = src_DEM.read().transpose(1, 2, 0)
    profile_DEM = src_DEM.profile

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RasterioDeprecationWarning: Creation keywords stored on datasets by Rasterio versions < 1.0b1 will always be ignored in version 1.0. You may opt in to ignoring them now by setting RIO_IGNORE_CREATION_KWDS=TRUE in your environment.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
img_DEM.shape

(10135, 8727, 1)

In [10]:
subsample(INPUT_PATH, OUTPUT_PATH, IMAGE_DEM.split(sep='.')[0], FeatureCollection.read(INPUT_PATH + GEOJSON)[0])

313037.0867231503 6449308.249773085 313655.60598492646 6449926.769034862
5111 5112


## Resample to the spatial resolution of 50 cm, save as BandCollection (separate image files for each channel and mask) plus geojsons.

In [11]:
def image_resize(input_path, img_name, px_size = 0.5):
    with rasterio.open(input_path + img_name + '.tif') as src:
        img = src.read()[0]
        profile = src.profile
        
    img_resize = cv.resize(img, None, fx=(profile['transform'][0]/px_size), fy=(-profile['transform'][4]/px_size), interpolation=cv.INTER_NEAREST)
    
    profile.update(transform=rasterio.Affine(px_size, 0.0, profile['transform'][2], 0.0, -px_size, profile['transform'][5]), 
               width = img_resize.shape[1], height = img_resize.shape[0])
        
    return img_resize, profile

def image_save(output_path, img, img_name, profile):
    with rasterio.open(output_path + img_name + '.tif', 'w', **profile) as dst:
        dst.write(img, 1)

In [12]:
RED_resize, profile_resize = image_resize(OUTPUT_PATH, 'RED', 0.5)
GRN_resize, _ = image_resize(OUTPUT_PATH, 'GRN', 0.5)
BLU_resize, _ = image_resize(OUTPUT_PATH, 'BLU', 0.5)

image_save(OUTPUT_PATH_RESIZE_05, RED_resize, 'RED_resize', profile_resize)
image_save(OUTPUT_PATH_RESIZE_05, GRN_resize, 'GRN_resize', profile_resize)
image_save(OUTPUT_PATH_RESIZE_05, BLU_resize, 'BLU_resize', profile_resize)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RasterioDeprecationWarning: Creation keywords stored on datasets by Rasterio versions < 1.0b1 will always be ignored in version 1.0. You may opt in to ignoring them now by setting RIO_IGNORE_CREATION_KWDS=TRUE in your environment.
  after removing the cwd from sys.path.


In [13]:
RED_resize, profile_resize = image_resize(OUTPUT_PATH, 'RED', 0.31)
GRN_resize, _ = image_resize(OUTPUT_PATH, 'GRN', 0.31)
BLU_resize, _ = image_resize(OUTPUT_PATH, 'BLU', 0.31)

image_save(OUTPUT_PATH_RESIZE_03, RED_resize, 'RED_resize', profile_resize)
image_save(OUTPUT_PATH_RESIZE_03, GRN_resize, 'GRN_resize', profile_resize)
image_save(OUTPUT_PATH_RESIZE_03, BLU_resize, 'BLU_resize', profile_resize)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RasterioDeprecationWarning: Creation keywords stored on datasets by Rasterio versions < 1.0b1 will always be ignored in version 1.0. You may opt in to ignoring them now by setting RIO_IGNORE_CREATION_KWDS=TRUE in your environment.
  after removing the cwd from sys.path.
